In [1]:
%load_ext autoreload
%autoreload 3

import gconcord as cc
import numpy as np

# import importlib
# importlib.reload(cc)

# true precision matrix
omega = np.zeros((3,3), order="C")
omega[0, 1] = 2.1
omega[1, 2] = -2.1
omega += omega.T
np.fill_diagonal(omega, 3)

# true covariance matrix
sigma = np.linalg.inv(omega)

# observations
data = np.random.multivariate_normal([0]*3, sigma, 500)

# sample covariance matrix
S = np.cov(data, rowvar=False)

In [2]:
# omega_hat, hist = cc.pyccista(S, 0.5, maxitr=1000)
omega_hat, hist = cc.pyccista(S, 0.5, maxitr=1000)
print("omega_hat:\n %s\n" % (str(omega_hat)))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("relative subg norm: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))

omega_hat:
 [[ 1.16616066  0.78917198  0.        ]
 [ 0.78917198  1.15987502 -0.77511828]
 [ 0.         -0.77511828  1.15536188]]

hist: (inner_iter_count, delta_subg, objective)
 [[5.00000000e+00 1.40865209e-05 1.83567229e+00]
 [1.00000000e+00 2.68817717e-05 1.83567229e+00]
 [5.00000000e+00 9.39526252e-06 1.83567229e+00]]

iterations: 85
relative subg norm: 9.395263e-06
objective: 1.835672e+00


In [3]:
sigii = np.diag(omega_hat.diagonal())
np.matmul(np.matmul(np.sqrt(sigii), omega_hat), np.sqrt(sigii))

array([[ 1.35993067,  0.91781773,  0.        ],
       [ 0.91781773,  1.34531005, -0.89728951],
       [ 0.        , -0.89728951,  1.33486107]])

In [4]:
np.linalg.inv(S)

array([[ 2.99330661,  2.07286801,  0.07751084],
       [ 2.07286801,  2.94655482, -2.00235647],
       [ 0.07751084, -2.00235647,  2.91586512]])

In [5]:
omega

array([[ 3. ,  2.1,  0. ],
       [ 2.1,  3. , -2.1],
       [ 0. , -2.1,  3. ]])

In [6]:
cc.ccista(S, 0.5, maxitr=1000).todense()

[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]


TypeError: ccista(): incompatible function arguments. The following argument types are supported:
    1. (arg0: numpy.ndarray[numpy.float64[m, n], flags.writeable], arg1: numpy.ndarray[numpy.float64[m, n], flags.writeable], arg2: float, arg3: int) -> scipy.sparse.csc_matrix[numpy.float64]

Invoked with: array([[  8.70620509, -11.77858763,  -8.31991723],
       [-11.77858763,  16.57152956,  11.69295435],
       [ -8.31991723,  11.69295435,   8.59379483]]), array([[0. , 0.5, 0.5],
       [0.5, 0. , 0.5],
       [0.5, 0.5, 0. ]]), 0.0, 1e-05, 1000, 1